In [18]:
import pandas as pd

types_dict = {"postal code": str, "rmse": float, "mae": float, "mpe": float, "parameter": float}

one_room = pd.read_csv("parameter_tuning_results/one-room-parameter-tuning-results.csv", index_col=[0], dtype=types_dict)
two_room = pd.read_csv("parameter_tuning_results/two-room-parameter-tuning-results.csv", index_col=[0], dtype=types_dict)
three_more_room = pd.read_csv("parameter_tuning_results/three_more-room-tuning-results.csv", index_col=[0], dtype=types_dict)
terrace_houses = pd.read_csv("parameter_tuning_results/terrace-parameter-tuning-results.csv", index_col=[0], dtype=types_dict)

In [19]:
one_room.head()

,postal code,rmse,mae,mpe,parameter
0,00100,957.562990,769.826597,0.089937,0.100
1,00120,128.818695,128.818695,0.014106,0.100
2,00140,63.859369,50.898271,0.005548,0.001
3,00150,580.522151,530.418388,0.057463,0.001
4,00170,561.856938,422.115598,0.047666,0.010


In [ ]:
import statistics

#plot_data = pd.DataFrame({"postal code": postal_codes_having_last_4_quarters, "rmse":rmse_list, "mae":mae_list, "mpe": mpe_list })

def show_results(data, mae_values, rmse_values, mpe_values,postal_code="postal code"):
    avg_rmse = statistics.mean(rmse_values)
    avg_mae = statistics.mean(mae_values)
    avg_mpe = statistics.mean(mpe_values)
    med_rmse = statistics.median(rmse_values)
    med_mae = statistics.median(mae_values)
    med_mpe = statistics.median(mpe_values)
    fig = data.plot.bar(x=postal_code, subplots=True, figsize=(200,100))
    plt.rcParams.update({'font.size': 22})
    print('Average MAE: %.3f' % avg_mae)
    #print('Average RMSE: %.3f' % avg_rmse)
    print('Average MPE: %.3f' % avg_mpe)
    print('Median MAE: %.3f' % med_mae)
    #print('Median RMSE: %.3f' % med_rmse)
    print('Median MPE: %.3f' % med_mpe)
    data.describe()
    
"""show_results(plot_data, mae_list, rmse_list, mpe_list)
print(len(rmse_list))"""